In [1]:
import requests
import json
import os
from pathlib import Path
import pandas as pd
path_logs = "../logs"

In [2]:
header_admin = {"identification": "admin:4dmin"}
localhost = "127.0.0.1"  # or "api"

### Get all users

In [3]:
### Get all users:
response = requests.get(url=f"http://{localhost}:8000/users",
                        headers=header_admin)
response.json()

{'detail': 'Not Found'}

In [11]:
response.json().keys()

dict_keys(['fdo', 'admin', 'policierA', 'policierB'])

### Get logs

In [4]:
# test sur tous les fichiers présents:
filenames = os.listdir(path_logs)
for filename in filenames:
    splitted = filename.split(".")[0]
    response = requests.post(url=f"http://{localhost}:8000/get_logs",
                            json={"name": splitted},
                            headers=header_admin)
    print(f"{splitted} loaded.")

f1_scores loaded.
preds_call loaded.
preds_labeled loaded.
preds_test loaded.
train loaded.
update_data loaded.


In [35]:
response = requests.post(url=f"http://{localhost}:8000/get_logs",
                        json={"name": "preds_test"},
                        headers=header_admin)
data_jsonl = response.json()
print(data_jsonl[1].keys())

dict_keys(['request_id', 'index', 'time_stamp', 'user_name', 'context', 'response_status_code', 'output_prediction', 'verified_prediction', 'model version', 'prediction_time', 'input_features'])


In [36]:
data_json = data_jsonl[0]
data_json.keys()

dict_keys(['request_id', 'index', 'time_stamp', 'user_name', 'context', 'response_status_code', 'output_prediction', 'verified_prediction', 'model version', 'prediction_time', 'input_features'])

In [37]:
def delecte_dict(data):
    dicts_to_delete = [
        "input_features", 
        "estimator_parameters", 
        "feature_importances"
    ]
    for entry in dicts_to_delete:
        if entry in data:
            del data[entry]
    return data

In [38]:
data_json = delecte_dict(data_json)
data_json.keys()


dict_keys(['request_id', 'index', 'time_stamp', 'user_name', 'context', 'response_status_code', 'output_prediction', 'verified_prediction', 'model version', 'prediction_time'])

In [27]:
data_json

{'request_id': '0096919261035789',
 'index': 8205,
 'time_stamp': '2024-05-17 14:04:07.086457',
 'user_name': 'admin',
 'context': 'test',
 'response_status_code': 200,
 'output_prediction': 0,
 'verified_prediction': 1,
 'model version': 'rdf_v1.0_shield.joblib',
 'prediction_time': 0.02099156379699707}

In [48]:
def jsonl_to_df(data_jsonl):
    new_jsonl = []
    for line in data_jsonl:
        data_json = delecte_dict(line)
        new_jsonl.append(data_json)

    # Ouvrir un fichier en mode écriture
    with open('data.jsonl', 'w') as file:
        for data in new_jsonl:
            # Convertir le dictionnaire en chaîne JSON et écrire dans le fichier
            file.write(json.dumps(data) + '\n')

    df = pd.read_json('data.jsonl', lines=True)

    return df

In [49]:
df = jsonl_to_df(data_jsonl)
df.head(3)

,request_id,index,time_stamp,user_name,context,response_status_code,output_prediction,verified_prediction,model version,prediction_time
0,96919261035789,8205,2024-05-17 14:04:07.086457,admin,test,200,0,1,rdf_v1.0_shield.joblib,0.020992
1,3879502289105825,8206,2024-05-17 14:06:00.890132,admin,test,200,1,1,rdf_v1.0_shield.joblib,0.018996
2,9686126869792342,8207,2024-05-17 14:06:01.762695,admin,test,200,0,0,rdf_v1.0_shield.joblib,0.018001
